# 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
train.info()
# Non-Null Count 체크하고 결측치 유무를 확인

# EDA
EDA(Exploratory Data Analysis, 탐색적 자료 분석)란, 수집한 데이터를 분석하기 전에 데이터의 특성을 관찰하고 이해하는 단계입니다.

## 첫번째, Target(y) 데이터에 대한 EDA

In [ ]:
# 파이썬 warning 무시
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트를 사용하기 위한 코드
fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

## 코드 작성          
train['next_arrive_time'].plot(figsize=(20,10), alpha=0.4)

plt.title('버스 운행시간')
plt.xlabel('인덱스')
plt.ylabel('운행시간')

plt.hlines([20, 250,600,2300], xmin=0, xmax=len(train), color='red')

plt.show()

In [ ]:
# 구간별 횟수 시각화

a = train[train['next_arrive_time']<20]
b = train[(train['next_arrive_time']>=20) & (train['next_arrive_time']<250)]
c = train[(train['next_arrive_time']>=250) & (train['next_arrive_time']<600)]
d = train[(train['next_arrive_time']>=600) & (train['next_arrive_time']<2300)]
e = train[(train['next_arrive_time']>=2300)]

plt.bar(x=['a', 'b', 'c', 'd', 'e'], height=[len(a), len(b), len(c), len(d), len(e)])   # b 구간이 가장 많게 나타남

plt.show()

In [ ]:
# b구간이 전체에서 차지하는 비율

x = ['b구간', '나머지']
y = [len(b)/len(train), (1-len(b)/len(train))]

plt.figure(dpi=150)

plt.title('버스 운행시간 구간 비율')

plt.pie(y, labels=x)
plt.legend()
plt.show()

In [ ]:
import matplotlib.patches as patches

x = train['distance']
y = train['next_arrive_time']

plt.figure(dpi = 150)

plt.title('거리 vs 운행시간')
plt.xlabel('거리')
plt.ylabel('운행시간')

plt.scatter(x,y, alpha = 0.3)

# 이상치 표시하기 - 직사각형
plt.gca().add_patch(
    patches.Rectangle(
        (0, 600),
        1000, 2600,               
        edgecolor = 'deeppink',
        fill=False,
    ))

plt.show()

# 두번째, Features(X) 데이터에 대한 EDA

어떤 컬럼에 대해서 하나의 고유한 값만 존재한다면, 그 컬럼은 식별 가능한 컬럼이라고 할 수 있습니다.

운행 정보(route_id, vh_id, route_nm) 컬럼 중 식별 가능한 컬럼이 존재한다면, 나머지는 분석에 사용하지 않아도 됩니다.

그럼 이러한 칼럼이 있는지 임의로 칼럼을 뽑아 살펴볼까요?

## route_id -> route_nm , route_nm -> route_id

In [ ]:
print(train[train['route_id'] == 405136001]['route_nm'].unique())
print(train[train['route_nm'] == '360-1']['route_id'].unique())

1:1 대응 관계였던 route_id, route_nm과 달리 vh_id는 1:n, 일대다 관계입니다.

그렇다면 반대로도 실험해 볼까요?

vh_id를 기준으로 하면 route_id와 route_nm의 고윳값을 구할 수 있네요.

이는 vh_id가 route_id와 route_nm보다 더 많은 정보를 포함하고 있음과 동시에 식별 가능한 컬럼임을 의미합니다.

    즉, vh_id를 알면 route_id, route_nm을 알 수 있습니다.

In [ ]:
def check(text:str):
    if (len(train[train['vh_id'] == text]['route_id'].unique()) != 1) | (len(train[train['vh_id'] == text]['route_nm'].unique()) != 1):
        return True
    else :
        return False

temp = list(map(check, train['vh_id'].unique()))
set(temp)

# 데이터 처리

앞 단계에서 확인한 데이터의 특징을 바탕으로 전처리 작업을 진행해 보도록 합시다!

## Feature Engineering
1. 값에 숫자 부여

In [ ]:
my_list = list(train['route_nm'].unique()) + list(train['now_station'].unique()) + list(train['next_station'].unique())
my_dict = {text : i for i, text in enumerate(my_list)}
my_dict

def transform_df(df_raw):
    df = df_raw.copy()
    df[['route_nm', 'now_station', 'next_station']] = df[['route_nm', 'now_station', 'next_station']].applymap(lambda x: my_dict[x])
    df['now_arrive_time'] = df['now_arrive_time'].map(lambda x: int(x[:2]))
    return df

train = transform_df(train)
test = transform_df(test)

2. 이상치 제거
3. 데이터 분리 (train -> train/validation)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.1, shuffle=False)

#데이터 shape 확인
print(f"X_train.shape : {X_train.shape}")
print(f"y_train.shape : {y_train.shape}")
print(f"X_valid.shape : {X_valid.shape}")
print(f"y_valid.shape : {y_valid.shape}")

# 모델링

## Random Forest
랜덤 포레스트는 한마디로, 훈련 과정에서 만들어진 다수의 의사 결정 나무로부터 분류된 결과를 집계해 최종적으로 분류된 데이터, 또는 평균 예측치를 출력하는 모델입니다.

랜덤 포레스트에서는 이 앙상블 기법 중에서도 배깅을 이용합니다!

### 배깅(Bagging)
배깅(Bagging)은 'Bootstrap + Aggregating'의 합성어인데요.

여기서 부트스트랩(Bootstrap)이란, 표본 분포를 구하기 위해 데이터를 여러 번 복원 추출(랜덤 샘플링)하는 방법입니다.
이 때, 중복을 허용하기 때문에 단일 데이터가 여러 번 선택될 수도 있습니다.

배깅은 이러한 부트스트랩을 통해서 다양한 데이터셋을 만들고, 이를 학습시킨 모델을 모으는(Arregating) 방법입니다.

즉, 랜덤 포레스트에서 배깅은 모든 의사 결정 나무가 학습 데이터 세트에서 임의로 하위 데이터 세트를 추출하는 과정을 말하는 것이라 이해해 주시면 됩니다.

예를 들어 학습 데이터 세트에 총 1000개의 행이 있다고 하면, 임의로 행을 100개씩 선택해서 의사 결정 나무를 만드는 것입니다.

### 배깅 속성 (Bagging Feature)
의사 결정 나무를 만들 때는 사용될 속성(feature)들을 제한하여 각 나무들에 다양성을 줘야 합니다.

따라서 모든 속성(feature)들에서 임의로 일부를 선택하고, 그중 정보 획득량이 가장 높은 것을 기준으로 데이터를 분할합니다.

만약 데이터 세트에 n개의 속성이 있는 경우, n 제곱근 개수만큼 무작위로 선택하는 것이 일반적입니다. (A rule of thumb)

예를 들어 총 25개의 속성이 있으면 그중에서 n 제곱근인 5개의 속성만 뽑아서 살펴본 후, 정보 획득량이 가장 높은 걸 기준으로 데이터를 분할하는 것입니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X, y)
# 학습된 모델을 이용해 결괏값 예측 후 상위 10개의 값 확인
predict = model.predict(test[features])
print('----------------------예측된 데이터의 상위 10개의 값 확인--------------------\n')
print(predict[:10])

### 하이퍼 파라미터란?

하이퍼 파라미터(hyper parameter)는 모델링할 때 사용자가 직접 세팅해 주는 값을 뜻합니다.

머신러닝 모델을 쓸 때 사용자가 직접 세팅해야 하는 값이 상당히 많은데, 그 모든 값이 다 하이퍼 파라미터입니다.

ex)

- batch_size: 배치 크기
- (training) epochs: 반복 학습 횟수
- optimizer: 옵티마이저
- learning rate: 학습률
- activation functions: 활성화 함수

| 하이퍼 파라미터는 정해진 최적 값이 존재하지 않으며, 설정에 따라 성능에 큰 차이를 보이기도 합니다.

| 하이퍼 파라미터와 혼용되곤 하는 **파라미터(parameter, 매개변수)** 는 학습 과정에서 생성되는 변수를 말합니다. 다시 말해서 사용자가 임의로 설정하는 값이 아닙니다.

사용자가 직접 설정하면 하이퍼 파라미터, 모델 혹은 데이터에 의해 결정되면 파라미터입니다.

그럼, RandomForestRegressor 모델의 하이퍼 파라미터를 튜닝하여 성능을 높여 봅시다!

n_estimators : 결정 트리의 개수(defalut=100)
criterion : 분할된 것(split)의 품질을 측정하는 기능
max_depth : 트리의 최대 깊이
min_samples_split : 노드를 분할하기 위한 최소한의 샘플 데이터 수
이외에도 랜덤 포레스트 모델에는 다양한 하이퍼 파라미터가 존재합니다.

이번 하이퍼 파라미터 튜닝에서는 n_estimators와 criterion을 다룹니다.

In [ ]:
model = RandomForestRegressor(n_estimators=100, criterion='mse', random_state=42)

### 평가
이제 모델을 이용해 예측한 결괏값과 실제값을 비교하여 모델을 평가하고 성능을 확인해 볼까요?

평가산식 : RMSE
이번 버스 운행 시간 예측 프로젝트에서 사용할 평가산식은 RMSE(Root Mean Squared Error)입니다.

Image
RMSE는 오류 지표를 실제값과 유사한 단위로 다시 변환하여 해석을 쉽게 하며, MAE보다 특이치에 강합니다.

예측하고자 하는 변수, 즉 target이 수치형인 회귀 문제에 사용됩니다.

<회귀 모델을 평가하는 평가 지표>

- MAE(Mean Absolute Error): 모델의 예측값과 실제값 차이의 절대값의 평균
- MSE(Mean Squared Error): 모델의 예측값과 실제값 차이의 제곱의 평균
- RMSE(Root Mean Squared Error): MSE에 루트를 씌운 것

RMSE 점수도 sklearn 패키지를 이용해 구할 수 있습니다.

## XGBoost
XGBoost는 Extreme Gradient Boosting의 약자입니다.

이 XGBoost라는 모델을 알기 위해선, 먼저 Boosting(부스팅)에 관해 이해하실 필요가 있습니다!

### Boosting

Boosting은 한마디로 말해, 순차적으로 모델의 정확도를 높이는 방법입니다.

Boosting에서는 먼저 전체 학습 데이터에서 일부를 선택한 하위 데이터 세트와 이를 학습할 첫 번째 모델을 만듭니다.

그리고 첫 번째 모델이 잘 학습하지 못한 부분을 반영해서 두 번째 데이터 세트와 모델을 만들고,
이런 과정을 반복해서 점진적으로 모델의 정확도를 높입니다.

이러한 Boosting 기법을 이용하여 구현한 알고리즘은 Gradient Boost가 대표적인데요.
이 Gradient Boost 알고리즘을 병렬 학습이 지원되도록 구현한 것이 바로 XGBoost 입니다.

Regression, Classification 문제를 모두 지원하며, 성능과 자원 효율이 좋아서 자주 사용되는 알고리즘이라고 할 수 있습니다.

### 하이퍼 파라미터
- objective: 목적함수
    * 'reg:squarederror'는 오차 제곱입니다.
- n_estimators: 트리 수
- tree_method: gpu 사용
- eval_set: 성능 평가를 수행할 데이터 세트
- eval_metric: 조기 종료를 위한 평가 지표
- early_stopping_rounds: 조기 종료 조건, 평가 지표가 향상될 수 있는 반복 횟수
- verbose: 학습 결과 출력 조건

이외에도 XGBRegressor 모델에는 다양한 하이퍼 파라미터가 존재합니다.

※ Early Stopping (조기 중단) 기능

    GBM의 경우 n_estimators에 지정된 횟수만큼 학습을 끝까지 수행하지만, XGB의 경우 오류가 더 이상 개선되지 않으면 수행을 중지합니다. 만약 n_estimators 를 200으로 설정하고, 조기 중단 파라미터 값을 50으로 설정하면, 1부터 200회까지 부스팅을 하다가 50회를 반복하는 동안 학습 오류가 감소하지 않으면 더 이상 부스팅을 진행하지 않고 종료합니다. (ex. 100회에서 학습 오류 값이 0.8인데 101~150회 반복하는 동안 예측 오류가 0.8보다 작은 값이 하나도 없으면 부스팅을 종료) 조기 중단 기능은 불필요한 학습 시간을 단축시켜 준다는 장점이 있습니다. 하지만 이 조기 중단 값을 급격하게 줄이게 되면 모델 성능이 향상될 여지가 있음에도 불구하고 학습이 조기 중단되는 경우가 발생할 수 있습니다.

import xgboost as xgb

# 1. 모델 정의
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators = 3000)

# 2. 모델 학습
model.fit(X_train,y_train, eval_set=[(X_valid,y_valid)],
          eval_metric = 'rmse',
          early_stopping_rounds=10,
          verbose= True
          )

# 3. 예측
# predict() 메소드 이용
predict = model.predict(test[features])


# 예측값 시각화
plt.plot(predict)
plt.show()